In [26]:
from selenium import webdriver
from PIL import Image as pil
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
category, date, page = 100, 20191121, 1

url = "https://news.naver.com/main/list.nhn?\
mode=LSD&mid=sec&sid1={}&date={}&page={}".format(category, date, page)

In [5]:
driver = webdriver.Chrome()
driver.get(url)

In [8]:
articles = driver.find_elements_by_css_selector(".list_body > ul > li")
len(articles)

20

In [9]:
title = articles[0].find_element_by_css_selector("dt:not(.photo) > a").text
link = articles[0].find_element_by_css_selector("dt:not(.photo) > a")\
.get_attribute("href")
title, link

('"필사즉생" 황교안, 단식 전 영양제 논란',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=052&aid=0001368505')

In [15]:
response = requests.get(link)
# 아래와 같이 오류가 발생하면 다음 셀의 코드를 실행해주세요.

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [22]:
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36"
}
response = requests.get(link, headers=headers)
dom = BeautifulSoup(response.content, "html.parser")

In [23]:
content = dom.select_one("#articleBodyContents").text\
.replace("\n", "").replace("\'", "").replace("- ", "").split("{}")[1].strip()
content

'■ 진행 : 이종구 앵커■ 출연 : 김병민 경희대 객원교수, 양지열 변호사* 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다.[앵커] 자유한국당 황교안 대표가 청와대 앞 분수대 광장과 국회를 오가면서 출퇴근 단식농성을 이어가려 있습니다. 이런 가운데 황교안 대표가 단식 돌입 전날 영양제를 맞았다는 보도가 나와서 논란도 되고 있습니다. 여당과 다른 야당 비판 수위를 계속 높이고 있습니다. 오늘 나이트포커스 양지열 변호사, 김병민 경희대 객원교수 두 분과 함께합니다. 안녕하세요?먼저 주제어 영상부터 보시겠습니다. 황교안 대표 세 가지의 명분을 앞세워서 죽기를 각오하고 단식투쟁을 이틀째 벌이고 있습니다. 일단은 황교안 대표가 죽기를 각오한다. 이건 본인의 얘기지만요. 잠은 국회 앞 텐트에서 자고 투쟁만 청와대 앞에서 하는 거죠? [양지열] 그렇습니다. 그래서 처음에는 청와대에서 애초에 계속해서 단식을 하겠다라고 밝혔었는데 청와대 앞에서 1인 시위 자체도 불가능하고 거기에 텐트를 설치하는 것은 경호 문제 때문에 불가능하다라는 그런 통지를 받으면서 국회 앞으로 불가피하게 텐트가 있으니까 그쪽으로 옮긴 건데 사실 왜 청와대 앞에서 해야 되느냐를 놓고 처음부터 말이 있었죠. 왜냐하면 지금 주장하고 있는 것들이 세 가지인데 첫 번째가 지소미아 철폐를 하지 말라는 것이고 나머지 두 가지는 패스트트랙하고 연동형 비례대표제 관련인데. 그럼 패스트트랙과 연동형 비례대표제는 국회에서 지금 진행되고 있는 일이지 않습니까?그러면 그 부분을 청와대에서 나서서 뭐라고 할 수 있는 게 아니잖아요. 그러니까 삼권분립에 어긋나는 주장을 어떻게 보면 하고 계신 그런 상황이고. 나경원 원내대표와의 어떤 얘기를 통해서 원내에서 이 일을 풀어나가야 하는 게 맞는데 저게 지금 청와대 앞에 가서 단식할 일이냐는 근본적인 질문부터 할 수밖에 없어요.게다가 하나, 지소미아 관련된 부분은 청와대가 주도하는 것이지만 결국 지소미아 종료 문제는 내일 사이에서

In [61]:
def get_articles(category, date, page):
    url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1={}&date={}&page={}".format(category, date, page)
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36"
    }
    articles = driver.find_elements_by_css_selector(".list_body > ul > li")
    datas = []
    for article in articles:
        title = article.find_element_by_css_selector("dt:not(.photo) > a").text
        link = article.find_element_by_css_selector("dt:not(.photo) > a").get_attribute("href")
        response = requests.get(link, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        try:
            
            content = dom.select_one("#articleBodyContents").text.replace("\n", "").replace("\'", "").replace("- ", "").split("{}")[1].strip()
        except:
            content = dom.select_one("#articeBody").text.replace("\n", "")
        datas.append({
            "title": title,
            "link": link,
            "content": content,
            "category": category,
        })
    driver.quit()
    return pd.DataFrame(datas)

In [62]:
categories = {
    100: "정치",
    101: "경제",
    102: "사회",
    103: "생활/문화",
    104: "세계",
    105: "IT/과학",
}

In [63]:
dfs = []
date = 20191121
for category in categories:
    print(categories[category], end=" ")
    for page in range(1, 6):
        print(page, end=" ")
        df = get_articles(category, date, page)
        dfs.append(df)
    print()
articles_df = pd.concat(dfs, ignore_index=True)
articles_df.tail()

정치 1 2 3 4 5 
경제 1 2 3 4 5 
사회 1 2 3 4 5 
생활/문화 1 2 3 4 5 
세계 1 2 3 4 5 
IT/과학 1 2 3 4 5 


,category,content,link,title
595,105,“고령화가 급속히 진행되면서 섬유증 치료제 시장은 면역항암제 다음으로 환자들의 수요...,https://news.naver.com/main/read.nhn?mode=LSD&...,"이봉용 넥스트젠바이오사이언스 대표 ""황반변성 치료제, 점안제로 개발 중…내년 美 임상"""
596,105,(서울=연합뉴스) 황창규 KT 회장과 마크 베니오프 세일즈포스 CEO가 지난 19일...,https://news.naver.com/main/read.nhn?mode=LSD&...,드림포스 2019 행사장 둘러보는 황창규 KT회장
597,105,(서울=연합뉴스) 황창규 KT 회장과 마크 베니오프 세일즈포스 CEO가 지난 19일...,https://news.naver.com/main/read.nhn?mode=LSD&...,드림포스 2019 행사장 둘러보는 황창규 KT회장
598,105,제공 | 하이콘[스포츠서울 이상훈 기자] 하이콘(HYCON)이 블록체인 기반 채굴형...,https://news.naver.com/main/read.nhn?mode=LSD&...,"하이콘, ‘크립토 워리어즈’와 파트너십 체결"
599,105,[머니투데이 김지영 기자] [[2019 모바일 컨퍼런스]이필성 샌드박스네트워크 대표...,https://news.naver.com/main/read.nhn?mode=LSD&...,누구나 유튜버가 되는 시대…성공하는 '유튜버'의 조건은?


In [64]:
articles_df.to_csv("article.csv", index=False, encoding="utf-8-sig")